# Import the dataset

In [1]:
import numpy as np
import pandas as pd
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchaudio
from torchvision.transforms import functional as F
from torch.utils.data import Dataset
from google.colab import drive

# Mount Google Drive to access data
drive.mount('/content/drive')

# Defining the local path to Google Drive
audio_path = '/content/drive/MyDrive/genres_original'

# Check if the directory exists
if not os.path.exists(audio_path):
    raise FileNotFoundError(f'Directory not found: {audio_path}')

# Define a custom dataset class to load audio files
class AudioSpectrogramDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.file_paths = []
        self.labels = []

        # Iterate through each class folder in the root directory
        for label in os.listdir(root):
            class_folder = os.path.join(root, label)
            if os.path.isdir(class_folder):
                # Iterate through each audio file in the class folder
                for file in os.listdir(class_folder):
                    file_path = os.path.join(class_folder, file)
                    self.file_paths.append(file_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        audio_path = self.file_paths[idx]
        label = int(self.labels[idx])  # Convert label to integer

        # Load audio file
        waveform, sample_rate = torchaudio.load(audio_path)

        # Convert audio waveform to spectrogram
        spectrogram = torchaudio.transforms.Spectrogram()(waveform)

        # Apply any additional transformations
        if self.transform:
            spectrogram = self.transform(spectrogram)

        return spectrogram, label

# Define transformations for the spectrograms (e.g., resizing)
spectrogram_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize spectrogram to a common size
    transforms.ToTensor()           # Convert spectrogram to a PyTorch tensor
])

# Create an instance of the AudioSpectrogramDataset
audio_dataset = AudioSpectrogramDataset(root=audio_path, transform=spectrogram_transform)

# Create a DataLoader for batching and shuffling the data
batch_size = 32
data_loader = DataLoader(audio_dataset, batch_size=batch_size, shuffle=True)

# Cross-check the number of classes present in the dataset
num_classes = len(set(audio_dataset.labels))
class_names = audio_dataset.labels

print(f"Number of classes: {num_classes}")
print(f"Name of classes: {class_names}")



Mounted at /content/drive
Number of classes: 10
Name of classes: ['reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reggae', 'reg

# RNN with LSTM

In [2]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import LSTM, Linear, CrossEntropyLoss
from torch import optim, nn
from torch.nn.utils.rnn import pad_sequence
from audioread.exceptions import NoBackendError

class GenreDataset(Dataset):
    """Custom dataset class for music genre classification."""

    def __init__(self, data_path, labels, n_mfcc=20):
        self.data_path = data_path
        self.labels = labels
        self.n_mfcc = n_mfcc
        self.file_list = []
        for genre, folder in data_path.items():
            for filename in os.listdir(folder):
                self.file_list.append((os.path.join(folder, filename), labels[genre]))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path, label = self.file_list[idx]
        mfccs = self.extract_features(file_path)
        if mfccs is None:
            mfccs = torch.zeros(self.n_mfcc, 1)
        return mfccs, label

    def extract_features(self, file_path):
        """Loads audio file, extracts MFCCs."""
        try:
            y, sr = librosa.load(file_path)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=self.n_mfcc)
            return torch.Tensor(mfccs)
        except (FileNotFoundError, NoBackendError) as e:
            print(f"Error extracting features from {file_path}: {e}")
            return None

# Define paths and load data
data_path = {
    "blues": "/content/drive/MyDrive/genres_original/blues",
    "classical": "/content/drive/MyDrive/genres_original/classical",
    "country": "/content/drive/MyDrive/genres_original/country",
    "disco": "/content/drive/MyDrive/genres_original/disco",
    "hiphop": "/content/drive/MyDrive/genres_original/hiphop",
    "jazz": "/content/drive/MyDrive/genres_original/jazz",
    "metal": "/content/drive/MyDrive/genres_original/metal",
    "pop": "/content/drive/MyDrive/genres_original/pop",
    "reggae": "/content/drive/MyDrive/genres_original/reggae",
    "rock": "/content/drive/MyDrive/genres_original/rock",
}

labels = {genre: i for i, genre in enumerate(data_path.keys())}

# Create the dataset
dataset = GenreDataset(data_path, labels)

# Split data into training and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])


def collate_fn(batch):
    # Sort batch by sequence length (descending order) for pack_padded_sequence
    batch.sort(key=lambda x: x[0].size(1), reverse=True)

    # Separate sequences and labels
    sequences, labels = zip(*batch)

    # Determine the length of the shortest sequence in the batch
    min_length = min([seq.size(1) for seq in sequences])

    # Truncate longer sequences to match the length of the shortest sequence
    sequences_truncated = [seq[:, :min_length] for seq in sequences]

    # Pad sequences to have the same length
    sequences_padded = pad_sequence(sequences_truncated, batch_first=True)

    return sequences_padded, torch.tensor(labels)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)



class RNNModel(nn.Module):
    """RNN model with LSTM layer for genre classification."""

    def __init__(self, input_dim, hidden_dim, num_classes):
        super(RNNModel, self).__init__()
        self.lstm = LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = Linear(hidden_dim, num_classes)

    def forward(self, x):
        # Pass data through LSTM layer
        lstm_out, _ = self.lstm(x)
        # Get the last output from the LSTM layer
        out = lstm_out[:, -1, :]
        # Pass the output to the fully connected layer
        out = self.fc(out)
        return out

# Define model hyperparameters
input_dim = 20  # Assuming 20 MFCC coefficients
hidden_dim = 128
num_classes = len(data_path)  # Number of unique genres

# Create model instance
model = RNNModel(input_dim, hidden_dim, num_classes)

# Define loss function and optimizer
criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Train phase
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        optimizer.zero_grad()

        # Ensure input data has the correct shape
        data = data.permute(0, 2, 1)  # Permute dimensions to (batch_size, num_channels, sequence_length)

        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Calculate average training loss
    avg_train_loss = train_loss / len(train_loader)


# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        # Ensure input data has the correct shape
        data = data.permute(0, 2, 1)  # Permute dimensions to (batch_size, num_channels, sequence_length)

        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f" The Test accuracy of the model is: {accuracy* 100:.2f}%")



Epoch 1/50


<ipython-input-2-87770e12c2ca>:37: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Epoch 2/50


<ipython-input-2-87770e12c2ca>:37: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Epoch 3/50


...

Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Epoch 49/50


<ipython-input-2-87770e12c2ca>:37: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Epoch 50/50


<ipython-input-2-87770e12c2ca>:37: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
 The Test accuracy of the model is: 54.00%


# RNN with GAN

In [3]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import LSTM, Linear, CrossEntropyLoss
from torch import optim, nn
from torch.nn.utils.rnn import pad_sequence
from audioread.exceptions import NoBackendError
from torch.nn.utils.rnn import pad_sequence

# Define the GAN model for audio generation
class GANModel:
    def __init__(self):
        # Define your GAN model architecture and training procedure here
        pass

    def generate_audio_sample(self):
        # Generate a synthetic audio sample using the trained generator
        pass

# Define the dataset class for music genre classification
class GenreDataset(Dataset):
    def __init__(self, data_path, labels, n_mfcc=20):
        self.data_path = data_path
        self.labels = labels
        self.n_mfcc = n_mfcc
        self.file_list = []
        for genre, folder in data_path.items():
            for filename in os.listdir(folder):
                self.file_list.append((os.path.join(folder, filename), labels[genre]))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path, label = self.file_list[idx]
        mfccs = self.extract_features(file_path)
        if mfccs is None:
            mfccs = torch.zeros(self.n_mfcc, 1)
        return mfccs, label

    def extract_features(self, file_path):
        try:
            y, sr = librosa.load(file_path)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=self.n_mfcc)
            return torch.Tensor(mfccs)
        except (FileNotFoundError, NoBackendError) as e:
            print(f"Error extracting features from {file_path}: {e}")
            return None

# Define paths and load data
data_path = {
    "blues": "/content/drive/MyDrive/genres_original/blues",
    "classical": "/content/drive/MyDrive/genres_original/classical",
    "country": "/content/drive/MyDrive/genres_original/country",
    "disco": "/content/drive/MyDrive/genres_original/disco",
    "hiphop": "/content/drive/MyDrive/genres_original/hiphop",
    "jazz": "/content/drive/MyDrive/genres_original/jazz",
    "metal": "/content/drive/MyDrive/genres_original/metal",
    "pop": "/content/drive/MyDrive/genres_original/pop",
    "reggae": "/content/drive/MyDrive/genres_original/reggae",
    "rock": "/content/drive/MyDrive/genres_original/rock",
}

labels = {genre: i for i, genre in enumerate(data_path.keys())}

# Create the dataset
dataset = GenreDataset(data_path, labels)

# Split data into training and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])


def collate_fn(batch):
    # Exclude None values from the batch
    batch = [item for item in batch if item is not None]

    # If all items are None, return an empty batch
    if len(batch) == 0:
        return None, None

    batch.sort(key=lambda x: x[0].size(1), reverse=True)
    sequences, labels = zip(*batch)
    min_length = min([seq.size(1) for seq in sequences])
    sequences_truncated = [seq[:, :min_length] for seq in sequences]
    sequences_padded = pad_sequence(sequences_truncated, batch_first=True)
    return sequences_padded, torch.tensor(labels)


# Create data loaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Define the RNN model for genre classification
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(RNNModel, self).__init__()
        self.lstm = LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = Linear(hidden_dim, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]
        out = self.fc(out)
        return out

# Define model hyperparameters
input_dim = 20  # Assuming 20 MFCC coefficients
hidden_dim = 128
num_classes = len(data_path)

# Create model instance
model = RNNModel(input_dim, hidden_dim, num_classes)

# Define loss function and optimizer
criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define GAN model for audio generation
gan_model = GANModel()

# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Train phase
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        optimizer.zero_grad()
        data = data.permute(0, 2, 1)
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()





# Generate synthetic audio samples using GAN and augment the training dataset
num_synthetic_samples = len(train_data)
synthetic_samples = [gan_model.generate_audio_sample() for _ in range(num_synthetic_samples)]
augmented_train_data = train_data + synthetic_samples

# Create a new DataLoader for the augmented training dataset
augmented_train_loader = DataLoader(augmented_train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Retrain the model using the augmented training dataset
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Train phase
    model.train()
    train_loss = 0.0
    for data, target in augmented_train_loader:
        optimizer.zero_grad()
        data = data.permute(0, 2, 1)
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Calculate average training loss
    avg_train_loss = train_loss / len(augmented_train_loader)
    print(f"Train Loss: {avg_train_loss*100:.2f}%")

# Evaluate the model after augmentation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data = data.permute(0, 2, 1)
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f"The Test accuracy of the model is: {accuracy* 100:.2f}%")


Epoch 1/50


<ipython-input-3-120425fb046b>:46: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Epoch 2/50


<ipython-input-3-120425fb046b>:46: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Epoch 3/50


...

Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Train Loss: 6.82%
Epoch 49/50


<ipython-input-3-120425fb046b>:46: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Train Loss: 6.37%
Epoch 50/50


<ipython-input-3-120425fb046b>:46: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error extracting features from /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: 
Train Loss: 6.62%
The Test accuracy of the model is: 49.00%
